## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 160.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 268.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 331.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 287.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.2.5 which is incompatible.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.

In [ ]:
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
blosc2 3.3.1 requires numpy>=1.26, but you have numpy 1.25.2

In [ ]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,
    n_gpu_layers=43,
    n_ctx=4096,  # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [ ]:
def response(query,max_tokens=256,temperature=0,top_p=0.95,top_k=50):
    model_output = lcpp_llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control: Identify and address the source of infection, if possible. This may involve surgical intervention, such as drainage of an abscess or debridement of necrotic tissue.\n3. Fluid resuscitation: Administer intravenous fluids to maintain adequate blood pressure and organ perfusion. The goal is to achieve a mean arterial pressure (MAP) of at least 65 mmHg and a central venous oxygen saturation (ScvO2) of greater than 70%.\n4. Vasopressor support: 

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain is typically sharp, localized in the lower right quadrant of the abdomen, and worsens with movement or pressure on the area.\n2. Loss of appetite: People with appendicitis may lose their appetite due to abdominal discomfort.\n3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially in more advanced cases.\n4. Fever: A fever of 100.4°F (38°C) or higher may be present.\n5. Constipation or diarrhea: Some people with appendicitis experience constipation, while others have diarrhea.\n6. Inability to pass gas: Flatulence is often difficult or impossible to pass due to the inflammation and pressure on the area.\n7. Rebound tenderness: When the doctor presses on'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, and eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications.\n\nThere are several treatments that have been shown to be effective in addressing sudden patchy hair loss:\n\n1. Corticosteroids: These are anti-inflammatory drugs that can help reduce inflammation and suppress the immune system's attack on the hair follicles. They can be applied topically or taken orally, depending on the severity of the condition.\n2. Minoxidil: This is a medication that has been shown to promote hair growth in some people with alopecia areata. It works by incre

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as swelling, seizures, pain, or infections. For example, corticosteroids may be used to reduce brain swelling, and anticonvulsants may be given to prevent seizures.\n3. Rehabilitation: Rehabilitation is an essential part of the recovery process for TBI patients. This may include physical therapy to help regain mobility, occupational therapy to learn new skills or adapt to disabilities, speech therapy to improv

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or low blood pressure. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, a makeshift sling, or a hiking pole to immobilize the injured leg. Be careful not to apply too much pressure on the fracture site.\n4. Provide warmth and shelter: Keep the person warm and shield them from the elements to prevent hypothermia.\n5. Seek medical help: If possible, call for emergency medical services or arrange for transportation to a hospital.\n\nOnce you've ensured the person's safety and stability, here are some general treatment steps an

## Question Answering using LLM with Prompt Engineering

In [ ]:
system_prompt = "You are a highly knowledgeable and reliable medical assistant trained in the latest clinical guidelines, medical research, and pharmaceutical data. Your role is to support healthcare professionals by providing fast, accurate information. Prioritize clarity, reliability, and clinical relevance in your responses.Focus areas include: Symptom recognition and differential diagnosis. Medication details including trade and generic names, dosages, and contraindications. Evidence-based treatment plans and first-line therapies. Protocols for specialty care (e.g., endocrine disorders).Critical care procedures and emergency response protocols. Ensure your responses reference current standards of care and are suitable for use by licensed medical professionals. If unsure or if more information is needed for accuracy, state the limitations. The user input is:"


### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


"\nI'd be happy to help you with that question!\nSepsis is a life-threatening condition caused by a dysregulated response to infection. In a critical care unit, managing sepsis involves a multidisciplinary approach aimed at addressing the underlying infection and supporting organ function. Here are some key steps in the sepsis management protocol:\n1. Early recognition and diagnosis: Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to identify patients who may have sepsis.\n2. Fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion, aiming for a mean arterial pressure (MAP) of 65 mmHg or higher and a central venous oxygen saturation (ScvO2) of 70% or higher.\n3. Antibiotic therapy: Start broad-spectrum antibiotics as soon as possible based on the patient's clinical presentation, culture results, and local guidelines.\n4. Source control: Identify and address the source of infection, such as rem

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input = system_prompt+"\n"+ "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. Common symptoms include:\n1. Sudden and persistent pain in the lower right abdomen, often starting around the navel and then moving to the right side.\n2. Loss of appetite and feeling sick to your stomach (nausea).\n3. Vomiting.\n4. Fever, which may be low-grade at first but can rise as high as 101°F (38.3°C) or more.\n5. Constipation or diarrhea.\n6. Abdominal swelling and tenderness.\n7. Inability to pass gas or have a bowel movement.\n\nAppendicitis cannot be cured via medicine alone, as the inflammation can lead to rupture of the appendix if left untreated. If the appendix ruptures, it can cause peritonitis, a serious infection in the abdominal cavity that requires immediate surgical intervention. The standard treatment for appendicitis is an appendectomy, which involves removing the inflamed'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input = system_prompt+"\n"+ "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that results in the sudden loss of hair from specific areas of the scalp or other parts of the body. The exact cause of alopecia areata is unknown, but it's believed to be related to a problem with the immune system.\n\nEffective treatments for addressing sudden patchy hair loss include:\n\n1. Corticosteroids: Topical and injectable corticosteroids are commonly used to treat alopecia areata. They help reduce inflammation and suppress the immune response, allowing hair regrowth.\n2. Immunomodulators: Drugs like minoxidil and anthralin can be used to stimulate hair growth by modifying the immune response.\n3. Dithranol: A topical medication that works by reducing inflammation and promoting hair regrowth.\n4. JAK inhibitors: Oral medications like tofacitinib and upadacitinib have shown promising results in clinical trials for treating alopecia areata.\n5. Hair transplantation: In severe cases, hair tran

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input = system_prompt+"\n"+ "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


'\nBased on current standards of care, the treatment for a person with a brain injury depends on the severity and location of the injury. Here are some common interventions:\n1. Initial assessment and stabilization: This includes managing airway, breathing, and circulation, controlling bleeding, and preventing further injury.\n2. Medications: Depending on the specific symptoms, medications may be prescribed to manage pain, reduce inflammation, prevent or treat seizures, improve cognitive function, or address other complications.\n3. Rehabilitation: This includes physical therapy, occupational therapy, speech and language therapy, and cognitive rehabilitation to help the person regain lost skills and functions.\n4. Surgery: In some cases, surgery may be necessary to remove hematomas, repair skull fractures, or relieve pressure on the brain.\n5. Nutritional support: The person may require nutritional support through a feeding tube if they are unable to eat or drink adequately.\n6. Psycho

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input = system_prompt+"\n"+ "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\nBased on current standards of care, here's an evidence-based approach to managing a leg fracture sustained during a hiking trip:\n1. Assess the severity of the injury: Determine if it is an open or closed fracture, and assess for signs of nerve or vessel damage such as numbness, tingling, or pulselessness.\n2. Immobilize the affected limb: Use a splint or a sling to prevent further movement and reduce pain. If necessary, apply a tourniquet above the fracture site to control bleeding, but only if there is significant hemorrhage and as a last resort due to the risk of complications.\n3. Provide pain relief: Administer appropriate analgesics such as acetaminophen or nonsteroidal anti-inflammatory drugs (NSAIDs) for pain management. Consider opioids if other pain medications are not effective.\n4. Transport the patient to a healthcare facility: Arrange for emergency medical services or transportation to a hospital, especially if there are signs of nerve or vessel damage, significant ble

## Data Preparation for RAG

### Loading the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
manual_pdf_path = "/content/drive/My Drive/UTA_AI_ML/GenAI/medical_diagnosis_manual.pdf"

In [ ]:
pdf_loader = PyMuPDFLoader(manual_pdf_path)

In [ ]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [ ]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
laksh.g20@gmail.com
629V8BTFXN
eant for personal use by laksh.g20@gm
shing the contents in part or full is liable 

Page Number : 2
laksh.g20@gmail.com
629V8BTFXN
This file is meant for personal use by laksh.g20@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 

#### Checking the number of pages

In [ ]:
len(manual)

4114

### Data Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [ ]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
len(document_chunks)

8456

In [ ]:
document_chunks[0].page_content

'laksh.g20@gmail.com\n629V8BTFXN\neant for personal use by laksh.g20@gm\nshing the contents in part or full is liable'

In [ ]:
document_chunks[2].page_content

'Table of Contents\n1\nFront    ................................................................................................................................................................................................................\n1\nCover    .......................................................................................................................................................................................................\n2\nFront Matter    ...........................................................................................................................................................................................\n53\n1 - Nutritional Disorders    ...............................................................................................................................................................\n53\nChapter 1. Nutrition: General Considerations    ...........................................................................................

In [ ]:
document_chunks[3].page_content

"305\nChapter 25. Drugs & the Liver    ................................................................................................................................................\n308\nChapter 26. Alcoholic Liver Disease    ....................................................................................................................................\n314\nChapter 27. Fibrosis & Cirrhosis    ............................................................................................................................................\n322\nChapter 28. Hepatitis    ..................................................................................................................................................................\n333\nChapter 29. Vascular Disorders of the Liver    .....................................................................................................................\n341\nChapter 30. Liver Masses & Granulomas    ...........................................

In [ ]:
i = 5
document_chunks[i]

Document(page_content='742\n7 - Dermatologic Disorders    ....................................................................................................................................................\n742\nChapter 71. Approach to the Dermatologic Patient    .......................................................................................................\n755\nChapter 72. Principles of Topical Dermatologic Therapy    ............................................................................................\n760\nChapter 73. Acne & Related Disorders    ...............................................................................................................................\n766\nChapter 74. Bullous Diseases    .................................................................................................................................................\n771\nChapter 75. Cornification Disorders    .......................................................................

There are some overlaps in the chunking as we have set the overlap size while defining the splitter

### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Vector Database

In [ ]:
report = 'Medical_Manual'
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    collection_name=report
)

### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 6}
)

### System and User Prompt Template

In [ ]:
qna_system_message = """
You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context.

"""
qna_user_message_template = """
Conider the following ###Context and ###Question
###Context
{context}

###Question
{question}
"""

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = lcpp_llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Based on the context, the protocol for managing sepsis in a critical care unit includes:\n1. Aggressive fluid resuscitation until CVP reaches 8 mm Hg (10 cm H2O) or PAOP reaches 12 to 15 mm Hg.\n2. Administration of antibiotics based on suspected source, clinical setting, and previous culture results.\n3. Surgical excision of infected or necrotic tissues and drainage of pus.\n4. Supportive care.\n5. Normalization of blood glucose with a continuous'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


"Based on the context provided, the common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia; after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left"

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,top_k=20)

"Based on the context provided, sudden patchy hair loss, also known as alopecia areata, can be treated with corticosteroids, topical anthralin, minoxidil, or in severe cases, systemic corticosteroids. The cause of this condition is believed to be an autoimmune disorder affecting genetically susceptible individuals exposed to unclear environmental triggers. It's important to note that treatment for alopecia areata may lead to spontaneous regrowth of hair, but it can also become chronic or spread diffusely. Additionally, risk factors for chronic"

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Based on the context provided, the following treatments are recommended for a person who has sustained a physical injury to brain tissue:\n1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.\n2. Surgery to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.\n3. Maintaining adequate brain perfusion and oxygenation and preventing complications in the first few days after the injury.\n4. Rehabilitation services, particularly in cognitive and'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Based on the context provided, the following information can be derived:\n1. Fractures are cracks in bones that cause pain, swelling, ecchymosis, crepitation, deformity, and abnormal motion.\n2. Symptoms of a fracture include pain, swelling, ecchymosis, crepitation, deformity, and abnormal motion.\n3. Immobilization is necessary for treating fractures to prevent further injury and facilitate healing.\n4. Joints proximal and distal to the injury should be immobilized.\n5.'

### Fine-tuning

Query 1: What is the protocol for managing sepsis in a critical care unit?

We will try to increase the temperature and also  decreasing top_k or top_p slightly to provide more relavant results.If my responses are too repetitive or lack diversity, we would consider increasing top_k or top_p slightly but this is not the case.

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,temperature=0.5,top_k=25)

Llama.generate: prefix-match hit


'Based on the context, the protocol for managing sepsis in a critical care unit includes:\n1. Aggressive fluid resuscitation with 0.9% saline until CVP reaches 8 mm Hg (10 cm H2O) or PAOP reaches 12 to 15 mm Hg.\n2. Monitoring of vital signs, urine output, and renal function.\n3. Administration of antibiotics based on suspected source, clinical setting, knowledge or suspicion of causative organisms, sensitivity patterns common to that specific inpatient unit, and previous culture results.'

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,temperature=0.5,top_k=25)

Llama.generate: prefix-match hit


"The common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia; after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point. However, these classic findings appear in less than 50% of patients. Pain may not be localized, particularly in infants and children. Tenderness may be diffuse or absent, and bowel movements are usually less frequent"

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,temperature=0.5,top_k=25)

Llama.generate: prefix-match hit


'Based on the context provided, sudden patchy hair loss, also known as alopecia areata, is typically an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers. The effective treatments for this condition include topical or intralesional corticosteroids, topical anthralin, minoxidil, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). The treatment is usually most effective when it is initiated early in the course of the'

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,temperature=0.5,top_k=25)

Llama.generate: prefix-match hit


'Based on the provided context, the following treatments are recommended for a person who has sustained a physical injury to brain tissue:\n1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.\n2. Surgery to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.\n3. Supportive care, including preventing systemic complications due to immobilization, providing good nutrition, preventing pressure ulcers, and physical therapy to prevent limb contractures.'

What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,temperature=0.5,top_k=25)

Llama.generate: prefix-match hit


'The context provides information on the general approach to treating fractures, including supportive measures, local wound care, and sometimes delayed surgery. It also mentions immobilization as a treatment method for fractures, which can be done using either a cast or a splint. The context also discusses the importance of good hygiene and keeping the affected area dry in order to prevent infection.\n\nBased on this information, the necessary precautions for a person who has fractured their leg during a hiking trip would include immobilizing the injured limb using either a cast or a splint, keeping it dry, and'

## Output Evaluation

In [ ]:
groundedness_rater_system_message = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""

In [ ]:
relevance_rater_system_message = """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = lcpp_llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = lcpp_llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    response_2 = lcpp_llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=300, temperature=0.5)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5

Explanation: The answer is derived completely from the context as it summarizes the key points of the protocol for managing sepsis in a critical care unit as outlined in the provided text.

 Rating: 5

Explanation: The provided answer directly addresses the question by outlining the key steps in managing sepsis in a critical care unit, including fluid resuscitation, vasopressor use, oxygen therapy, and infection evaluation. The context provided in the text discusses various aspects of critical care medicine, including patient monitoring, blood tests, and supportive care. The answer incorporates several relevant details from the context, such as the importance of central venous pressure and pulmonary artery occlusive pressure targets during fluid resuscitation, the potential use of dopamine and norepinephrine as vasopress


Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=300, temperature=0.5)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5

Explanation: The answer is derived completely from the context provided, which includes detailed information about the approach to managing sepsis in a critical care unit, including fluid resuscitation, antibiotic administration, surgical intervention, supportive care, normalization of blood glucose, and corticosteroid therapy. The answer also mentions monitoring vital signs, fluid intake and output, urine output, and laboratory tests, which is consistent with the context's discussion on patient monitoring in ICUs. Additionally, the answer mentions activated protein C as an emerging therapy for severe sepsis and septic shock, which is discussed in the context. Overall, the answer accurately reflects the information presented in the context.

 Rating: 5

Explanation: The AI-generated answer directly addresses all the important aspects of managing sepsis in a critical care unit from the provided context, including fluid resuscitation, administration of antibiotics, surgical i

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=300, temperature=0.5)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5

Explanation: The answer is completely derived from the context provided, which discusses various treatments and possible causes for alopecia areata, a type of sudden patchy hair loss. The answer accurately identifies corticosteroids, anthralin, minoxidil, immunotherapy, and autoimmune disorders as relevant to this condition, based on the information presented in the context.

 Rating: 5

Explanation: The AI-generated answer directly addresses the key aspects of the question by providing a list of effective treatments for sudden patchy hair loss, which is a characteristic feature of alopecia areata. It also mentions the possible cause of this condition as an autoimmune disorder. The context provided in the text supports these statements and adds additional details about each treatment option. Therefore, the answer is fully relevant to the question based on the provided context.


Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",max_tokens=300, temperature=0.5)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 2

Explanation: The answer is derived from the context to a limited extent as it mentions the supportive care measures mentioned in the context. However, the answer does not go into detail about the specific treatments recommended for a person with a brain injury beyond these measures.

 Rating: 3

Explanation: The answer is moderately relevant as it correctly identifies the importance of supportive care in the initial treatment of traumatic brain injuries, including maintaining adequate brain perfusion and oxygenation, preventing complications, and providing physical therapy to prevent limb contractures. However, it does not directly address the question by listing specific treatments beyond these measures. The context does mention some treatments such as surgery for patients with more severe injury and rehabilitation services for those whose coma exceeds 24 hours, but these are not included in the answer.


Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=300, temperature=0.5)

print(ground,end="\n\n")
print(rel)

NameError: name 'generate_ground_relevance_response' is not defined

## Actionable Insights and Business Recommendations

# Comparison of Outputs and Evaluation:

**LLM (Without Context):**
Outputs: Generic, sometimes inaccurate, lacking specific medical information or protocols.

**RAG (with Context):**
Outputs: Significant improvement over LLM, more specific and accurate, but occasional struggles with synthesizing information and providing complete responses.

**Fine-tuned RAG:**
Outputs: Further improvements in response quality, better synthesis of information, more comprehensive and coherent answers, and increased focus and alignment with queries.

**Actionable Insights:**

**Context is Key**: We confirmed the important role of context in achieving accurate and relevant medical responses. RAG's has significant improvement over LLM.

**Fine-tuning Refines Performance**: Fine-tuning, particularly adjusting the temperature, further enhances the ability of RAG to synthesize information and generate focused answers, leading to higher groundedness and relevance scores.

Output Evaluation is Crucial: Analyzing groundedness and relevance scores provides valuable insights into the system's strengths and weaknesses, guiding further improvements.
Iterative Refinement is Necessary: Continuously monitoring and refining the RAG system based on evaluation results is essential for maintaining accuracy and relevance over time.
Human Oversight Remains Important: Despite improvements, human review is still crucial to validate outputs, especially in medical applications where safety and accuracy are paramount.
Business Recommendations:
Prioritize RAG-based Systems: For medical question-answering, prioritize the development and deployment of RAG-based systems over standalone LLMs due to their superior performance in groundedness and relevance.
Invest in Knowledge Base Enhancement: Continuously update and enhance the medical knowledge base used as context to ensure comprehensive and up-to-date information for the RAG system.
Fine-tune for Optimal Performance: Experiment with fine-tuning parameters like temperature and potentially top_k/top_p to optimize the system's ability to generate relevant and grounded responses.
Integrate Output Evaluation: Incorporate automated or human-based evaluation mechanisms to regularly assess the groundedness and relevance of the system's outputs.
Develop User Feedback Loops: Implement systems to collect feedback from medical professionals using the tool, enabling iterative improvements based on real-world usage and insights.
Prioritize Transparency and Explainability: Provide insights into the system's reasoning process and the sources used to generate answers to enhance trust and understanding among medical professionals.
Explore Personalization: Consider extending the RAG system to provide personalized medical recommendations by incorporating patient-specific data and medical history.
By combining insights from output comparison and evaluation, you can create a robust and reliable medical question-answering system that supports healthcare professionals in making informed decisions and delivering better patient care.

<font size=6 color='blue'>Power Ahead</font>
___